## Problem Set 5

Yewon Kim 

## 1. Estimating the Brock and Mirman (1972) model by GMM 

Data on ($c_t, k_t, w_t, r_t$) is given. We will use the following set of equations for estimation: 

$$(c_t)^{-1} -\beta E[r_{t+1}(c_{t+1})^{-1}] = 0$$

$$ c_t+ k_{t+1} - w_t - r_tk_t = 0 $$

$$ w_t-(1-\alpha)\exp(z_t)(k_t)^\alpha = 0 $$

$$ r_t - \alpha \exp(z_t)(k_t)^{\alpha-1} =0 $$

$$z_t = \rho z_{t-1} + (1-\rho)\mu + \epsilon_t $$ 
$$\mbox{where } E[\epsilon_t]=0$$

We have 4 parameters in total $(\alpha, \beta, \rho, \mu)$ to be estimated. 
$$\alpha, \beta \in (0,1),~~~ \mu >0, ~~~ \rho \in (-1,1)$$ 

In [1]:
# Import packages and load the data
import numpy as np
import numpy.random as rnd
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
cmap1 = matplotlib.cm.get_cmap('summer')
import pandas
# This next command is specifically for Jupyter Notebook
%matplotlib notebook

data = np.loadtxt("MacroSeries.txt", delimiter=",")

In [2]:
df = pandas.DataFrame(data)
## Renaming columns
df.columns = ['c', 'k', 'w', 'r']
df.head()

,c,k,w,r
0,1.067109e+07,8.040697e+06,1.059438e+07,0.954120
1,1.169487e+07,7.595072e+06,1.161079e+07,1.107009
2,9.292394e+06,8.323735e+06,9.225588e+06,0.802596
3,7.913165e+06,6.613792e+06,7.856274e+06,0.860176
4,8.140994e+06,5.632136e+06,8.082466e+06,1.039183


In [3]:
def current(x):
    if len(np.array(x).shape)== 1:
        return np.array(x[:-1])
    if len(np.array(x).shape) > 1:
        return np.array(x[:-1,:])
def future(x):
    if len(np.array(x).shape)== 1:
        return np.array(x[1:])
    if len(np.array(x).shape)> 1:
        return np.array(x[1:,:])


Within the GMMcrit function, I use the following property driven from equation (4):
$$\ z_t = \log(r_t) - \log(\alpha(k_t)^{\alpha-1}) = \log(r_t) - \log(\alpha)-(\alpha-1)\log(k_t)$$

In [39]:
def GMMcrit(params, *args):
    aa,bb,rr,mm = params
    W, data= args
    c, k,w,r = data
    
    z = np.log(r)-np.log(aa)-(aa-1)*np.log(k)
    
    m1 = np.mean(future(z) - rr*current(z) - (1-rr)*mm)
    m2 = np.mean((future(z)/(rr*current(z)+(1-rr)*mm)-1))
    m3 = np.mean((future(z)/(rr*current(z)+(1-rr)*mm)-1)*current(z))
    #m4 = np.mean(bb*future(r)*current(c)/future(c)-1)
    m4 = np.mean((bb*aa*np.exp(future(z))*(future(k)**(aa-1))*current(c)/future(c)-1))
    m5 = np.mean((bb*aa*np.exp(future(z))*(future(k)**(aa-1))*current(c)/future(c)-1)*current(w))
    
    mm = [m1,m2,m3,m4, m5]
    return np.dot(np.dot(mm, W), mm)

Check if the function works: 

In [40]:
p0 = np.array([0.1,0.1,0.1,2])
GMMcrit(p0, np.eye(5),(df.c, df.k, df.w, df.r))

87673094447636.688

In [41]:
a_init = 0.3
b_init = 0.9
r_init = 0.4
m_init = 0.7
params_init = np.array([a_init, b_init, r_init, m_init])
W = np.eye(5)
gmm_args = (W, (df.c, df.k, df.w, df.r))

minimizer_kwargs = dict(method="L-BFGS-B", bounds=((1e-10, 1-1e-10), (1e-10, 1-1e-10), (-1+1e-10,1-1e-10),(1e-10,None)),args = (gmm_args))
results_a = opt.basinhopping(GMMcrit, params_init, minimizer_kwargs = minimizer_kwargs)
results_a

                        fun: 1.8440214027919621e-05
 lowest_optimization_result:       fun: 1.8440214027919621e-05
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  5.42306066e-04,   1.02656389e+06,  -4.16748802e-02,
        -1.35525272e-12])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 240
      nit: 22
   status: 0
  success: True
        x: array([ 0.56512374,  0.99      ,  1.        ,  0.00321486])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 10
                       nfev: 14220
                        nit: 100
                          x: array([ 0.56512374,  0.99      ,  1.        ,  0.00321486])

In [42]:
alpha_GMM, beta_GMM, rho_GMM, mu_GMM = results_a.x

Let's see what the calculated moments look like:

In [45]:
def GMM_moments(params, *args):
    aa,bb,rr,mm = params
    W, data= args
    c, k,w,r = data
    
    z = np.log(r)-np.log(aa)-(aa-1)*np.log(k)
    
    m1 = np.mean(future(z) - rr*current(z) - (1-rr)*mm)
    m2 = np.mean((future(z)/(rr*current(z)+(1-rr)*mm)-1))
    m3 = np.mean((future(z)/(rr*current(z)+(1-rr)*mm)-1)*current(z))
    #m4 = np.mean(bb*future(r)*current(c)/future(c)-1)
    m4 = np.mean((bb*aa*np.exp(future(z))*(future(k)**(aa-1))*current(c)/future(c)-1))
    m5 = np.mean((bb*aa*np.exp(future(z))*(future(k)**(aa-1))*current(c)/future(c)-1)*current(w))
    
    mm = [m1,m2,m3,m4,m5]
    return mm

In [46]:
GMM_moments(results_a.x, W, (df.c, df.k, df.w, df.r))

[0.0013795811249966736,
 0.00027730553787768276,
 0.0013795811251345459,
 -3.6631525422420151e-10,
 -0.0038153410732603274]

## 2. Estimating the Brock and Mirman (1972) model by SMM

Steps for SMM are the following:

0) Draw T = 100 uniform draws outside the computation. 

1) Assume $z_1 = \mu$ and $k_1 = mean(k_t)$ from the data.

2) Convert 100 uniform draws to $\epsilon_t \sim N(0, \sigma)$ given $\sigma$, and simulate the full series of $z_t$ using equation (5)

3) Use the policy function for savings (10) recursively to solve for the entire $k_t$ series.
 
4) Using the simulated $k_t$ and $z_t$, use (3) to solve for the $w_t$ series and (4) to solve for the $r_t$ series.

5) Use the budge constraint (2) to solve for the $c_t$ series. 

In [5]:
## Step 0
T = 100
S = 1000
U = sts.norm.rvs(size=(T, S))

In [6]:
def SMMcrit(params, *args):
    aa,bb,rr,mm,ss = params
    #print(params)
    W, data = args
    c, k,w,r = data
    
    ## Step 1)
    
    z0 = mm
    k0 = np.mean(k)
    
    ## Step 2) & 3)
    eps = U*ss
    z_sim = np.array([rr*z0 + (1-rr)*mm + eps[0,:]])
    k_sim = np.array([[aa*bb*np.exp(z0)*((k0)**aa)]*S])
    
    for t in range(1,T):
        z_sim = np.vstack([z_sim, rr*z_sim[t-1,:]+(1-rr)*mm+eps[t,:]])
        k_sim = np.vstack([k_sim, aa*bb*np.exp(z_sim[t-1,:])*((k_sim[t-1,:])**aa)])

    
    #print(k_sim.shape)
        
    ## Step 4)
    w_sim = (1-aa)*np.exp(z_sim)*(k_sim**aa)
    r_sim = aa*np.exp(z_sim)*(k_sim**(aa-1))
    
    ## Step 5)
    c_sim = rr*current(k_sim) + current(w_sim) - future(k_sim)
    c_sim = np.vstack([c_sim,rr*k_sim[T-1,:]+w_sim[T-1,:]]) ### Assuming the last period's saving to be 0 
    #print(c_sim.shape)
    
    ## Creating model moments
    m1 = np.mean(c_sim)
    m2 = np.mean(k_sim)
    m3 = np.mean(np.var(c_sim,axis=0))
    m4 = np.mean(np.var(k_sim,axis=0))
    m5 = np.mean(sts.spearmanr(c_sim, k_sim, axis=0)[0])
    m6 = np.mean(sts.spearmanr(current(k_sim), future(k_sim), axis=0)[0])
    
    ## Creating data moments
    dd = np.array([np.mean(c), np.mean(k), np.var(c), np.var(k), sts.spearmanr(c,k)[0], sts.spearmanr(current(k), future(k))[0]])
    mm = np.array([m1, m2, m3, m4, m5, m6])
    
    err = (mm-dd)/dd
    #print(err)
    return np.dot(np.dot(err, W), err)
    

In [7]:
p0 = np.array([0.56512374,  0.99,  1.  ,  0.00321486 ,0.2])
SMMcrit(p0, np.eye(6),(df.c, df.k, df.w, df.r))

5.6879960318010543

In [8]:
params_init = np.array([0.56512374,  0.98,  0.98  ,  0.1, 0.01])
W = np.eye(6)
smm_args = (W, (df.c, df.k, df.w, df.r))

#minimizer_kwargs = dict(method="L-BFGS-B", bounds=((0.01, 0.99), (0.01, 0.99), (-0.99,0.99), (-0.5,1), (0.001,1)),args = (smm_args), options = {'eps':0.01})
#results_b = opt.basinhopping(SMMcrit, params_init, minimizer_kwargs = minimizer_kwargs)
results_b = opt.minimize(SMMcrit, params_init,method="L-BFGS-B", bounds=((0.01, 0.99), (0.01, 0.99), (-0.99,0.99), (-0.5,1), (0.001,1)),options = {'eps':1e-06},args = (smm_args))
results_b

      fun: 3.091362256879671
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -5.09344032e+00,   1.67941216e-04,  -3.37587736e-05,
         6.77320422e-05,   8.87359874e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 222
      nit: 21
   status: 0
  success: True
        x: array([ 0.99      ,  0.81397987, -0.43934325,  0.30985945,  0.001     ])

In [9]:
def SMM_moments(params, *args):
    aa,bb,rr,mm,ss = params
    W, data = args
    c, k,w,r = data
    
    ## Step 1)
    
    z0 = mm
    k0 = np.mean(k)
    
    ## Step 2) & 3)
    eps = sts.norm.ppf(U, scale = ss)
    z_sim = np.array([rr*z0 + (1-rr)*mm + eps[0,:]])
    k_sim = np.array([[aa*bb*np.exp(z0)*((k0)**aa)]*S])
    
    for t in range(1,T):
        z_sim = np.vstack([z_sim, rr*z_sim[t-1,:]+(1-rr)*mm+eps[t,:]])
        k_sim = np.vstack([k_sim, aa*bb*np.exp(z_sim[t-1,:])*((k_sim[t-1,:])**aa)])

    
    #print(k_sim.shape)
        
    ## Step 4)
    w_sim = (1-aa)*np.exp(z_sim)*(k_sim**aa)
    r_sim = aa*np.exp(z_sim)*(k_sim**(aa-1))
    
    ## Step 5)
    c_sim = rr*current(k_sim) + current(w_sim) - future(k_sim)
    c_sim = np.vstack([c_sim,rr*k_sim[T-1,:]+w_sim[T-1,:]]) ### Assuming the last period's saving to be 0 
    #print(c_sim.shape)
    
    ## Creating model moments
    m1 = np.mean(c_sim)
    m2 = np.mean(k_sim)
    m3 = np.mean(np.var(c_sim,axis=0))
    m4 = np.mean(np.var(k_sim,axis=0))
    m5 = np.mean(sts.spearmanr(c_sim, k_sim, axis=0)[0])
    m6 = np.mean(sts.spearmanr(current(k_sim), future(k_sim))[0])
    
    ## Creating data moments
    dd = np.array([np.mean(c), np.mean(k), np.var(c), np.var(k), sts.spearmanr(c,k)[0], sts.spearmanr(current(k), future(k))[0]])
    mm = np.array([m1, m2, m3, m4, m5, m6])
    
    err = (mm-dd)/dd
    print('Data moments:', dd)
    print('Model moments:', mm)
    return err
    

In [11]:
SMM_moments(results_b.x, W, (df.c, df.k, df.w, df.r))

Data moments: [  1.05208478e+07   7.47254456e+06   5.64243398e+12   2.81655086e+12
   8.75727573e-01   8.72962276e-01]
Model moments: [        nan         nan         nan         nan  0.97209356  0.96564849]


array([        nan,         nan,         nan,         nan,  0.11004105,
        0.10617437])